In [12]:
import numpy as np
import cv2

height, width = 501, 501

x = np.arange(width)
y = np.arange(height)
X, Y = np.meshgrid(x, y)

center_x, center_y = width // 2, height // 2

distance_values = np.sqrt((X - center_x) ** 2 + (Y - center_y) ** 2)

distance_map = cv2.normalize(distance_values, None, 0, 255, cv2.NORM_MINMAX)
distance_map = np.uint8(distance_map)  

def quantize_image(image, levels):
    step = 256 // levels  
    return (image // step) * step  

distance_map_16 = quantize_image(distance_map, 16)  
distance_map_4 = quantize_image(distance_map, 4)    
distance_map_1 = quantize_image(distance_map, 2)    

cv2.imshow("Original", distance_map)
cv2.imshow("16-Level", distance_map_16)
cv2.imshow("4-Level", distance_map_4)
cv2.imshow("Binary", distance_map_1)

cv2.waitKey()


-1

In [ ]:
import cv2
import numpy as np

image = cv2.imread('Maze.png', 0)

rows, cols = image.shape

padded_image = np.zeros((rows + 2, cols + 2), dtype=np.uint8)
padded_image[1:-1, 1:-1] = image

padded_image[padded_image >= 127] = 255
padded_image[padded_image < 127] = 0

result_matrix = np.zeros_like(padded_image, dtype=np.int32)
label_counter = 1

for i in range(1, rows + 1):
    for j in range(1, cols + 1):
        if padded_image[i, j] != 0:
            if padded_image[i - 1, j] == 0 and padded_image[i, j - 1] == 0:
                result_matrix[i, j] = label_counter
                label_counter += 1
            elif padded_image[i - 1, j] != 0 and padded_image[i, j - 1] == 0:
                result_matrix[i, j] = result_matrix[i - 1, j]
            elif padded_image[i - 1, j] == 0 and padded_image[i, j - 1] != 0:
                result_matrix[i, j] = result_matrix[i, j - 1]
            else:
                if result_matrix[i - 1, j] < result_matrix[i, j - 1]:
                    result_matrix[i, j] = result_matrix[i - 1, j]
                    result_matrix[i, j - 1] = result_matrix[i - 1, j]
                    label_counter -= 1
                elif result_matrix[i - 1, j] == result_matrix[i, j - 1]:
                    result_matrix[i, j] = result_matrix[i, j - 1]
                else:
                    result_matrix[i, j] = result_matrix[i, j - 1]
                    result_matrix[i - 1, j] = result_matrix[i, j - 1]

print('Number of objects:', np.amax(result_matrix))

def find_opening_label(result_matrix, row=0):
    for j in range(1, cols + 1):
        if result_matrix[1, j] != 0:  
            return result_matrix[1, j]
    return None 

def highlight_label(image, result_matrix, label):
    colored_image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)  
    for i in range(1, rows + 1):
        for j in range(1, cols + 1):
            if result_matrix[i, j] == label:
                colored_image[i - 1, j - 1] = (0, 0, 255) 
    return colored_image

opening_label = find_opening_label(result_matrix)
colored_image = highlight_label(image, result_matrix, opening_label)

cv2.imshow('Original Image', image)
cv2.imshow('Highlighted Opening', colored_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

Number of objects: 6


In [2]:
import cv2
import numpy as np
from collections import deque

binary_maze = cv2.imread("Maze.png", 0)
rows, cols = binary_maze.shape
colored_maze = cv2.cvtColor(binary_maze, cv2.COLOR_GRAY2BGR)

start = (0, np.where(binary_maze[0] == 255)[0][0])
end = (rows - 1, np.where(binary_maze[-1] == 255)[0][0])

def bfs(maze, start, end):
    queue = deque([([start], start)])  
    visited = set([start])

    while queue:
        path, (x, y) = queue.popleft()
        if (x, y) == end:
            return path  

        for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            nx, ny = x + dx, y + dy
            if 0 <= nx < rows and 0 <= ny < cols and maze[nx, ny] == 255 and (nx, ny) not in visited:
                queue.append((path + [(nx, ny)], (nx, ny)))
                visited.add((nx, ny))

path = bfs(binary_maze, start, end)
if path:
    for x, y in path:
        colored_maze[x, y] = (0, 0, 255)

cv2.imshow("Solved Maze", colored_maze)
cv2.waitKey()

-1